# Tutorial 5: Advanced Agent Techniques and Real-World Applications

In this tutorial, we'll explore advanced agent techniques in LangChain and apply them to create a sophisticated AI-powered research assistant.

In [9]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from langchain.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re

# Load environment variables
load_dotenv()

# Initialize Groq LLM
llm = ChatGroq(model_name="llama-3.1-70b-versatile")
embeddings = OllamaEmbeddings(model="all-minilm",base_url=os.getenv('OLLAMA_EMBEDDING_URL'))


## 1. Creating a Custom Agent with Specialized Capabilities

In [10]:
# Custom prompt template
template = """You are an AI research assistant designed to help with scientific literature analysis.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        match = re.search(r"Action: (.*?)[\n]*Action Input: (.*)", llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)


## 2. Implementing a Multi-Agent System

In [13]:
# Load and process research papers
loader = DirectoryLoader('research_papers', glob="*.txt" ,loader_cls=TextLoader)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(texts, embeddings)

# Define tools for the agents
search_tool = Tool(
    name="VectorStoreSearch",
    func=vectorstore.similarity_search,
    description="Searches the research papers for relevant information. Input should be a search query."
)

summarize_tool = Tool(
    name="Summarize",
    func=lambda x: llm(f"Summarize the following text in a concise manner: {x}"),
    description="Summarizes a given text. Input should be the text to summarize."
)

analyze_tool = Tool(
    name="Analyze",
    func=lambda x: llm(f"Analyze the following text and provide key insights: {x}"),
    description="Analyzes a given text and provides key insights. Input should be the text to analyze."
)

tools = [search_tool, summarize_tool, analyze_tool]

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

output_parser = CustomOutputParser()

llm_chain = LLMChain(llm=llm, prompt=prompt)

research_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=[tool.name for tool in tools]
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=research_agent, tools=tools, verbose=True)

## 3. Developing a Context-Aware Agent with Long-Term Memory

In [14]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

def converse_with_memory(input_text):
    response = conversation.predict(input=input_text)
    return response

memory_tool = Tool(
    name="ConversationMemory",
    func=converse_with_memory,
    description="Accesses the conversation history and provides context-aware responses. Input should be a question or statement."
)

tools.append(memory_tool)

context_aware_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=[tool.name for tool in tools]
)

context_aware_executor = AgentExecutor.from_agent_and_tools(agent=context_aware_agent, tools=tools, verbose=True)

C:\Users\Domenico\AppData\Local\Temp\ipykernel_46232\3984124537.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\Domenico\AppData\Local\Temp\ipykernel_46232\3984124537.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


## 4. Building a Real-World Application: AI-Powered Research Assistant

In [17]:
def research_assistant(query):
    print(f"Research Query: {query}\n")
    response = context_aware_executor.invoke(query)
    print(f"Research Assistant's Response: {response}\n")
    return response

# Test the AI-powered research assistant
research_queries = [
    "What are the main findings on climate change impacts in the research papers?",
    "Summarize the key methods used in analyzing climate data across the papers.",
    "Based on the previous findings, what are the most urgent areas for future research in climate change?",
    "Compare and contrast the conclusions from different papers on potential solutions to mitigate climate change."
]

for query in research_queries:
    research_assistant(query)

Research Query: What are the main findings on climate change impacts in the research papers?



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to search for relevant research papers on climate change impacts and then summarize the main findings. 

Action: VectorStoreSearch
Action Input: "climate change impacts"

Observation:[Document(metadata={'source': 'research_papers\\paper1.txt'}, page_content='# Climate Change Impacts on Global Ecosystems\n\n## Abstract\n\nThis paper reviews recent studies on the impacts of climate change on global ecosystems. We analyze data from various biomes, including tropical rainforests, coral reefs, and arctic tundra. Our findings indicate significant shifts in species distribution, phenology changes, and ecosystem degradation across multiple regions. We argue that urgent action is needed to mitigate these impacts and preserve biodiversity.\n\n## 1. Introduction\n\nClimate change, primarily driven by anthropogenic greenhouse

TypeError: Got unknown type S

## Summary and Key Takeaways

In this tutorial, we've explored advanced agent techniques in LangChain and applied them to create a sophisticated AI-powered research assistant:

1. **Custom Agent Creation**: We developed a specialized agent for scientific literature analysis, demonstrating how to tailor agents for specific domains.

2. **Multi-Agent System**: By combining multiple tools (search, summarize, analyze), we created a versatile system capable of handling complex research tasks.

3. **Context-Aware Agent with Long-Term Memory**: We implemented a conversation memory, allowing the agent to maintain context across multiple interactions and provide more coherent and relevant responses over time.

4. **Real-World Application**: We built an AI-powered research assistant that can analyze scientific papers, extract key information, and provide insights on complex topics like climate change.

Key Takeaways:
- Advanced agents can be tailored for specific domains and tasks, greatly enhancing their effectiveness.
- Combining multiple tools and agent types allows for the creation of powerful, multi-functional AI systems.
- Implementing memory and context awareness significantly improves the quality and coherence of agent responses over time.
- Real-world applications of these techniques can lead to powerful AI assistants capable of handling complex, domain-specific tasks.

Next Steps:
- Experiment with different combinations of tools and agent types for your specific use cases.
- Explore ways to further enhance the agent's memory and context understanding capabilities.
- Consider integrating external APIs or databases to expand the agent's knowledge and capabilities.
